In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import ast
import torch
import pickle as pkl

ingredients_db = pkl.load(open('../database/preprocessed_data_with_embeddings.pkl', 'rb'))

d:\TAMU\SEM 4\CNM\BrokenBasket\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
print(ingredients_db['embedding'][0])

[[-4.33827311e-01 -1.02978416e-01 -3.84010673e-01  2.02594340e-01
  -3.92109185e-01  1.44228190e-02  3.21285099e-01  6.17986202e-01
  -2.56583601e-01  2.37473980e-01  2.32314840e-01  4.66611624e-01
  -1.05638117e-01  6.46406591e-01  7.29735270e-02  1.97628111e-01
  -1.39657587e-01  5.92206955e-01  5.51026642e-01  1.35820374e-01
  -1.44485623e-01 -1.56013742e-01 -1.54998735e-01 -3.15942645e-01
   1.67503580e-03  3.19379121e-01  1.17442004e-01 -1.24848053e-01
   1.80009022e-01  2.09253699e-01 -1.91492051e-01  5.77393174e-01
  -2.38792628e-01  2.36459263e-02  5.31347275e-01 -1.13636367e-01
   1.53684288e-01 -9.60945711e-02 -1.51571929e-01  3.04041225e-02
   1.18682534e-01  1.75079316e-01  6.11857116e-01 -2.87639141e-01
  -1.59059882e-01 -5.06975614e-02 -2.70263243e+00 -5.34746945e-02
  -7.14718521e-01 -1.98801145e-01  3.97658311e-02 -1.16255336e-01
   1.68036539e-02 -1.61233753e-01  2.24435050e-02  5.90271592e-01
  -4.15637106e-01  5.75972915e-01  2.91786373e-01  5.84765263e-02
  -1.20358

In [2]:
temp_mp = {'Simple White Cake': [[280, 'all purpose flour'], [200, 'granulated sugar'], [100, 'unsalted butter'], [120, 'large eggs'], [10, 'vanilla extract']], 'Best Soft Chocolate Chip Cookies': [[225, 'all purpose flour'], [100, 'white sugar'], [50, 'light brown sugar'], [115, 'salted butter'], [60, 'large egg'], [40, 'vanilla'], [115, 'chocolate chips']], 'Best Homemade Brownies': [[190, 'all purpose flour'], [225, 'granulated sugar'], [80, 'cocoa powder'], [60, 'powdered sugar'], [115, 'dark chocolate chips'], [120, 'large eggs'], [60, 'canola oil'], [15, 'vanilla extract']]}

In [8]:
def cosine(a,b):
    print(type(a))
    print(type(b))
    # if a == 0 or b == 0:
    #     return 0
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
def check_meal_plan():
    print("---CREATE MEAL PLAN---")
    meal_plan = temp_mp
    current_calories = 0
    calorie_goal = 2000

    # encoder
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    for dish in meal_plan:
        for grams, ingredient in meal_plan[dish]:
            # vectorize ingredient
            print(ingredient)
            encoded_input = tokenizer(ingredient, max_length = 50, truncation=False, return_tensors='pt')
            print(encoded_input)
            encoded_input = pd.Series(encoded_input)
            # compare to db
            most_similar = (0,"")
            for index,item in enumerate(ingredients_db['embedding']):
                if cosine(item,encoded_input) > most_similar[0]:
                    most_similar = (cosine(item,encoded_input),index)

            # calculate calories
            current_calories += grams * ingredients_db['Data.Carbohydrate'][most_similar[1]]

    if current_calories > calorie_goal:
        return {"redo": True}
    else: 
        return {"redo": False, "current_calories": current_calories} 

check_meal_plan()

---CREATE MEAL PLAN---
all purpose flour
{'input_ids': tensor([[  101,  2035,  3800, 13724,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


ValueError: shapes (1,768) and (3,) not aligned: 768 (dim 1) != 3 (dim 0)